# Tensorflow

Tensorflow is an open source machine learning framework developed primarily by Google and released for a variety of languages.  We only focus on Python here, since that is the primary use of Tensorflow on ALCF systems.  For support for other modes, please contact support@alcf.anl.gov.

The tensorflow documentation is here:
https://www.tensorflow.org/

To get started with Tensorflow, import it:

In [1]:
import tensorflow as tf

2022-09-28 08:28:22.296653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Tensorflow basics

### `Tensor`
Tensorflow uses the concept of `Tensors` as data types, and supports a variety of operations on them.  This document is not meant to be a tensorflow tutorial - instead, this is meant to inform you of the core concepts of using Tensorflow on Polaris, assuming you have some familiarity with Tensorflow already.

You can learn more about tensors in detail here:
https://www.tensorflow.org/guide/tensor

### GPU Computing

Tensorflow supports GPU operations for a large set of mathematical operations on Tensors.  Generally, Tensorflow follows a "Compute Follows Data" model: when you operate on a tensor, the device used for computation will be the device where the data is resident.  For example:


In [2]:
# CPU Computing:

cpu_input_data = tf.random.uniform(shape=(2,5000,500))

# This runs on the CPU:

product = tf.linalg.matmul(cpu_input_data, cpu_input_data, transpose_a=True)

print(product.shape)
print(product.device)

(2, 500, 500)
/job:localhost/replica:0/task:0/device:CPU:0


### Getting access to Data

We'll cover the Data Pipelines more completely in a later presentation.  For now, we'll use the cifar10 dataset, available from tensorflow:

In [3]:
# # Trying to run this on a mac?  Try these lines if you get an SSL error
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


In [4]:
print(x_train.shape)
# This data has 50000 images, of size 32x32 pixels and 3 RGB colors

(50000, 32, 32, 3)


In [5]:
batch_data   = tf.convert_to_tensor(x_train[0:10], dtype=tf.float32) # Take the first 10 images
batch_labels = tf.convert_to_tensor(y_train[0:10], dtype=tf.float32) # first 10 labels

In [6]:
batch_labels.shape

TensorShape([10, 1])

### Machine Learning Models

Tensorflow is primarily developed as a machine learning framework, so may operations like convolution, dense layers, etc. are all well supported.

The easiest way to build a model is to use the Keras API for object-oriented model construction.  For example, building a few layers of a ResNet-likemodel can be done like so:

In [7]:
class ResidualBlock(tf.keras.Model):

    def __init__(self):
        # Call the parent class's __init__ to make this class functional with training loops:
        super().__init__()
        self.conv1  = tf.keras.layers.Conv2D(filters=16, kernel_size=[3,3], padding="same")
        self.conv2  = tf.keras.layers.Conv2D(filters=16, kernel_size=[3,3], padding="same")

    def call(self, inputs):
    
        # Apply the first weights + activation:
        outputs = tf.keras.activations.relu(self.conv1(inputs))
        # Apply the second weights:

        outputs = self.conv2(outputs)

        # Perform the residual step:

        outputs = outputs + inputs

        # Second activation layer:
        return tf.keras.activations.relu(outputs)



In [8]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        # Call the parent class's __init__ to make this class functional with training loops:
        super().__init__()
        
        self.conv_init = tf.keras.layers.Conv2D(filters=16, kernel_size=1)
        
        self.res1 = ResidualBlock()
        
        self.res2 = ResidualBlock()
        
        # 10 filters for each class:
        self.conv_final = tf.keras.layers.Conv2D(filters=10, kernel_size=1)
        
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        
    def call(self, inputs):
        
        x = self.conv_init(inputs)
        
        x = self.res1(x)
        
        x = self.res2(x)
        
        x = self.conv_final(x)
        
        return self.pool(x)

In [9]:
# Create a model:
model = MyModel()

In [10]:

output_data = model(batch_data)
print(output_data.shape)

(10, 10)


You can visualize your networks easily with Keras:

In [11]:
print(model.summary())

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  64        
                                                                 
 residual_block (ResidualBlo  multiple                 4640      
 ck)                                                             
                                                                 
 residual_block_1 (ResidualB  multiple                 4640      
 lock)                                                           
                                                                 
 conv2d_5 (Conv2D)           multiple                  170       
                                                                 
 global_average_pooling2d (G  multiple                 0         
 lobalAveragePooling2D)                                          
                                                          

## Using Tensorflow on Polaris

Tensorflow supports both GPU 

# Automatic Differentiation

The big advantage of the Machine Learning Frameworks is automatic differentiation.  Tensorflow supports automatic differentiation with the `GradientTape` syntax:

In [13]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()


In [14]:

with tf.GradientTape(persistent=True) as tape:
    logits = model(batch_data)
    loss = loss_function(batch_labels, logits)

Get the "normal" derivatives (with respect to parameters) with the tape:

In [15]:
grads = tape.gradient(loss, model.trainable_variables)

You can also get gradients of other components, by asking the tape to `watch` tensors:

In [16]:

with tf.GradientTape(persistent=True) as tape:
    tape.watch(batch_data)
    logits = model(batch_data)
    loss = loss_function(batch_labels, logits)

In [17]:
input_grads = tape.gradient(loss, batch_data)

In [18]:
input_grads.shape

TensorShape([10, 32, 32, 3])

### Reduced Precision